<a href="https://colab.research.google.com/github/cruz-marco/Recommender-DNC/blob/main/notebook_principal_anonimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.0 Requerimentos do ambiente

Foi criado um ambiente usando o venv para o desenvolvimento do modelo, este ambiente isolado usa Python versão 3.10.6 e as seguintes bibliotecas são necessárias para seu pleno funcionamento:

In [ ]:
%%writefile requirements.txt
pymongo==4.3.2
sklearn==1.1.3
numpy==1.23.4
mlflow==2.0.1

Overwriting requirements.txt


O código foi escrito seguindo as recomendações sugeridas nas documentações das bibliotecas, portanto, existem outras versões que podem ser usadas sem muito prejuízo esperado no funcionamento

O MLflow foi usado para fins de monitoramento. Nos foi informado que há o desenvolvimento de uma solução para este fim por parte da AoCubo, então a princípio, ele é opcional, encontra-se implementado, porém, desativado conforme os códigos a seguir.

# 2.0 Conexões com o banco de dados

## 2.1 Ponto de atenção

- A constante 'OUTPUT_NAME' faz referência à coleção onde a matriz de similaridade será salva e está sem um valor definido. Ela é usada pela função 'matrix_saver' que armazena toda a matriz de similaridade no banco de dados. Não testamos no ambiente de produção da AoCubo (apesar de termos acesso aos dados, jamais seria interessante atrapalhar o funcionamento do modelo vigente), mas pode ser facilmente configurado adicionando uma string com o nome da coleção.

- A função 'matrix_saver' está implementada no módulo de treinamento, entretanto desativada com um comentário. É aconselhável fazer uso dela em um ambiente de teste controlado, até para verificar o formato da matriz.

In [ ]:
%%writefile database.py
from pymongo import MongoClient


#String de conexão com o mongodb
CONNECT_STRING = None

#Nome da base de dados
DB_NAME = None #Nome da coleção que será consumida pelo treinamento do modelo
INPUT_NAME = 'data_portal' #Nome da coleção onde estão os dados de entrada.
OUTPUT_NAME = None #Nome da coleção onde será armazenada a matriz de similaridade.

client = MongoClient(CONNECT_STRING) #Criando o cliente de acesso
db = client[DB_NAME]#Selecionando o banco de dados

# Função para receber os dados em formato de lista.
def data_getter():
    return list(db[INPUT_NAME].find())

# Função para salvar a matriz no banco de dados.
def matrix_saver(matrix):
    return db[OUTPUT_NAME].insert_many(matrix.to_dict('records'))


Overwriting database.py


# 3.0 Funções de utilidades

Módulo que contém as funções de utilidades utilizadas no treinamento do modelo:

- 'pre_processer': É a função responsável pela verificação de valores duplicados, zerados ou faltantes; fazendo a verificação e remoção, caso hajam na massa a ser utilizada para o treinamento.

- 'matrix_mounter': A saída do modelo faz alusão somente ao índice do array que é consumido pelo NearestNeighbors. Portanto, esta função transtorma o array em Pandas DataFrame, nomeia as colunas e transtorma os índices em 'unit_id', referenciando os valores com o índice da massa de treinamento processada.

In [ ]:
%%writefile functions.py
from pymongo import MongoClient
import pandas as pd

def pre_processer(df):
  """
  Função criada para remover os dados nulos e faltantes antes do 
  treinamento do modelo.

  df: DataFrame montado a partir dos dados 'brutos' do MongoDB
  return: DataFrame sem zeros e valores faltantes.
  """
  #Eliminando valores duplicados
  if df.duplicated().sum() > 0:
    df = df.drop_duplicates()

  #Eliminando registros nulos
  if df.isna().sum().sum() > 0:
    df = df.dropna(how='any')

  #Eliminando registros com preço = 0
  if (df['price'] == 0).sum() > 0:
    df = df.drop(
        index=df[(df['price'] == 0)]\
        .index
    )
  return df


def matrix_mounter(neighbors, df_processed):
    """
    Função para montagem da matriz de semelhança usando os 
    índicies dos neighbors retornados pelo NearestNeighbors

    neighbors:  Array com os índices
    df_processed: Dataframe usado para o treinamento do modelo
    return: Matriz de similaridade processada com os ID's dos imóveis
    """
    s_matrix = pd.DataFrame(neighbors)\
    .apply(lambda x: df_processed.index[x])\
    .rename(columns={
        0: 'unit_id',
        1: 'neighbor_1',
        2: 'neighbor_2',
        3: 'neighbor_3',
        4: 'neighbor_4',
        5: 'neighbor_5',
        6: 'neighbor_6',
        7: 'neighbor_7',
    })\
    .set_index('unit_id')

    return s_matrix


Overwriting functions.py


# 4.0 Processamento de dados

Módulo onde acontece efetivamente o processamento de dados.

1. O DataFrame principal é montado a partir dos dados que estão na coleção 'data_portal' no MongoDB com as features selecionadas;
2. O conjunto de dados é verificado e tem os nulos, duplicados e sem valores eliminados;
3. É aplicada a transformação logarítmica em 'living_area' e 'price';
4. Os dados são escalonados usando o MinMaxScaler.

In [ ]:
%%writefile data_processing.py
#Preprocessamento de dados
from database import data_getter
from functions import pre_processer
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

#Definição das variáveis de interesse
FEATURES = ['bathrooms', 'bedrooms', 'living_area', 'latitude',
            'longitude', 'price', 'property_type_id']

#Montagem do dataframe principal
df = pd.DataFrame(
    data = data_getter()    
    ).set_index(
        'unit_id'
    )[FEATURES] 

df = pre_processer(df)

#Aplicando o log em living_area e em price
df[['living_area', 'price']] = np.log(df[['living_area', 'price']])

#Aplicando o MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df)
scaled_feats = scaler.transform(df)

df[FEATURES] = scaled_feats


Overwriting data_processing.py


# 5.0 Treinamento do modelo

Módulo principal do recomendador.

Neste módulo, que deve ser executado sempre que quisermos treinar o modelo, o NearestNeighbors consome a base de dados processada para o seu treinamento e, em seguida, a consome mais uma vez para criar a matriz de similaridade.

Existe uma implementação do MLflow pronta abaixo (basta ela ser descomentada), bem básica, mas que pode ajudar no monitoramento do modelo, enquanto a solução proprietária da AoCubo ainda não estiver pronta, caso achem necessário.

A função 'matrix_evaluator' retorna um DataFrame de amostragem aleatória de 15 imóveis, com suas respectivas características e distâncias de cosseno, para a avaliação do modelo. Conforme implementada abaixo, ela salva este teste em formato html na pasta 'test_logs', mas pode ser alterada conforme for de interesse.

In [ ]:
%%writefile model_training.py
#Treinamento do modelo
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from data_processing import df
from functions import matrix_mounter
from database import matrix_saver
from quality_eval import matrix_evaluator
import datetime
#import mlflow

run_name = f'KNN_Train_{str(datetime.datetime.now())}'
#mlflow.set_experiment(run_name)

#mlflow.start_run()

model = NearestNeighbors(n_neighbors=8, metric='cosine', algorithm='brute', 
                        n_jobs=-1)

#mlflow.log_params(model.get_params())


model.fit(df.to_numpy())

#Executando a predição. Retornando as distâncias e os índices dos vizinhos.
distances, neighbors = model.kneighbors(X=df.to_numpy())

#Monta a matriz de similaridade em um DataFrame, com os índices substituídos
#pelos 'unity_id' dos imóveis.
similarity_matrix = matrix_mounter(neighbors, df)

#Salva um dataframe em html na pasta test_logs para avaliação qualitativa do modelo. Pode ser desabilitada caso necessário.
matrix_evaluator(similarity_matrix, distances).to_html(f'./test_logs/{run_name}.html') 
#mlflow.log_artifact(f'./test_logs/{run_name}.html')


#matrix_saver(similarity_matrix) #Retirar este comentário somente depois de verificar todo o código referente ao banco de dados.

#mlflow.end_run()


Overwriting model_training.py


# 6.0 Avaliação da qualidade das recomendações

Módulo que contém somente a função de avaliação do modelo.

Esta função foi baseada numa função muito mais básica utilizada na sprint de 'Evaluation' do CRISP-DM, entretanto esta, muito mais robusta e complexa, retorna um dataframe com as recomendações e o imóvel de referência para uma amostragem aleatória de 15 imóveis amparados pela matriz de similaridade.

In [ ]:
%%writefile quality_eval.py
from database import data_getter
from data_processing import FEATURES
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.6f}'.format


def matrix_evaluator(sim_matrix, dists):
  #Buscando propriedades dos imóveis
  properties = pd.DataFrame(
                data = data_getter()    
                ).set_index(
                    'unit_id'
                )[FEATURES]\
                  .drop_duplicates()

  p_cols = list(properties.reset_index().columns) #Salvando a lista com nomes das colunas

  SAMPLE_SIZE = 15 #quantidade de registros para avaliação da amostra.

  #Buscando as distâncias para avaliação
  dists = pd.DataFrame(dists, index=sim_matrix.index)\
          .drop(columns=[0])\
          .rename(columns={
          1: 'neighbor_1',
          2: 'neighbor_2',
          3: 'neighbor_3',
          4: 'neighbor_4',
          5: 'neighbor_5',
          6: 'neighbor_6',
          7: 'neighbor_7'
          })

  #Gerando uma amostra aleatória
  samples = list(np.random.choice(sim_matrix.index, size=SAMPLE_SIZE))

  eval_df = pd.DataFrame()# Dataframe principal a ser retornado

  #Gerando uma linha em branco no dataframe
  blanked = pd.Series({k:v for (k,v) in zip(p_cols, 
                      ['---' for _ in range(len(p_cols))])})\
                      .to_frame().T
  blanked['distancies'] = '---'

  titles = pd.Series({k:v for (k,v) in zip(p_cols, p_cols)}).to_frame().T
  titles['distancies'] = 'distancies'

  #Loop para buscar as informações dos imóveis, cruzando com os IDs da matriz de similaridade.
  for sample in samples:
    #Busca as infos do imóvel de referência.
    ref = pd.DataFrame(properties.loc[sample]).T\
      .reset_index()\
      .rename(columns={'index': 'unit_id'})
    ref['distancies'] = 'reference'

    #Basca as infos das recomendações
    recs = properties.loc[sim_matrix.loc[sample].to_list()].reset_index()
    recs['distancies'] = dists.T[int(sample)].reset_index(drop=True)
    
    #Junta tudo neste DF
    joined = pd.concat([ref, recs]).reset_index(drop=True)

    #Acumula no DF principal de avaliação
    eval_df = pd.concat([eval_df, joined, blanked, titles])

  eval_df = eval_df.reset_index(drop=True)
  eval_df.drop(index=list(eval_df.iloc[-2:].index), inplace=True)

  return eval_df
  

Overwriting quality_eval.py
